In [1]:
import sys
sys.path.append("../")  # 确保根目录在 sys.path 中
from PIL import Image
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import src.postProcess as PostProcess
import src.particleGenerator as particleGenerator

import SimProfile

In [2]:

react_table3 = np.array([[[0.9, 2], [0.9, 3], [0.9, 4], [0.9, -4], [0.5, 7], [0.0, 0], [0.5, 8], [0.0, 0], [0.6, 10], [0.0, 0], [0.0, 0]],
                        [[0.5, 5], [0.0, 0], [0.0, 0], [0.0, 0], [0.5, 6], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0]],
                        [[1, -1], [1, -2], [1, -3], [1, -4], [1, -5], [1, -6], [1, -7], [1, -8], [1, -9], [1, -10], [0.0, 0]]])

# react_table3 = np.array([[[0.1, 2], [0.1, 3], [0.1, 4], [0.1, -4], [0.5, 7], [0.0, 0], [0.5, 8], [0.0, 0], [0.6, 10], [0.0, 0]],
#                         [[0.5, 5], [0.0, 0], [0.0, 0], [0.0, 0], [0.5, 6], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0]],
#                         [[0.27, -1], [0.27, -2], [0.27, -3], [0.27, -4], [0.27, -5], [0.27, -6], [0.27, -7], [0.27, -8], [0.27, -9], [0.27, -10]]])
color_names = ['dimgray', 'blue', 'red', 'green','cyan', 'black', 'white','yellow', 'brown', 'magenta', 'orange', 'purple', 'pink', 'gray']
labels = ['Si', 'SiF1', 'SiF2', 'SiF3', 'SiO', 'SiO2', 'SiOF', 'SiOF2', 'SiO2F', 'SiO2F2', 'mask']

# print(react_table3.shape)
# react_table = np.zeros((3, 11, 12))
react_table = np.zeros((react_table3.shape[0], react_table3.shape[1], react_table3.shape[1] + 1))

for i in range(react_table3.shape[0]):
    for j in range(react_table3.shape[1]):
        for k in range(react_table3.shape[2]):
            react_table[i, j, 0] = react_table3[i, j, 0]
            react_table[i, j, j+1] = -1
            react_chem =  int(np.abs(react_table3[i, j, 1]))
            if react_table3[i, j, 1] > 0:
                react_plus_min = 1
            elif react_table3[i, j, 1] < 0:
                react_plus_min = -1
            elif react_table3[i, j, 1] == 0:
                react_plus_min = 0
            react_table[i, j, react_chem] = react_plus_min

# print(react_table)
react_table_equation = react_table[:, :, 1:].copy()
react_table_equation = np.ascontiguousarray(react_table_equation, dtype=np.int32)

# react_type_table = np.array([[1, 1, 1, 4, 0], # 1: chlorination  # 0: no reaction  # 4: Themal etch
#                             [2, 2, 2, 2, 2], # 2 for physics and chemical sputtering
#                             [3, 3, 3, 3, 3]])

react_type_table = np.ones((4, 11), dtype=np.int32) # 1 for chemical transfer 
react_type_table[2, :] = 2 # 2 for physics
react_type_table[3, :] = 3 # 3 for redepo
react_type_table[0, 3] = 4 # 1 for chemical remove
react_type_table[2, -1] = 0 # 2 for no reaction for mask in test

print(react_type_table)

# react_prob_chemical = np.array([1.0, 1.0, 1.0, 1.0, 0.0])
# react_prob_chemical = np.array([0.50, 0.50, 0.50, 0.9, 0.0])

react_prob_chemical = react_table[:2, :, 0].copy()
react_prob_chemical = np.ascontiguousarray(react_prob_chemical, dtype=np.double)

# react_yield_p0 = np.array([0.30, 0.30, 0.30, 0.30, 0.30])
react_yield_p0 = np.array([0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30], dtype=np.double)

# film_eth = np.array([5, 5, 5, 5, 5], dtype=np.double)
film_eth = np.array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], dtype=np.double)

rn_coeffcients = np.array([[0.9423, 0.9434, 2.742, 3.026],
                            [0.9620, 0.9608, 2.542, 3.720],
                            [0.9458, 0.9445, 2.551, 3.735],
                            [1.046, 1.046, 2.686, 4.301]])

sputter_yield_coefficient = [0.3, 0.001, np.pi/4]

[[1 1 1 4 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 0]
 [3 3 3 3 3 3 3 3 3 3 3]]


In [3]:
react_prob_chemical

array([[0.9, 0.9, 0.9, 0.9, 0.5, 0. , 0.5, 0. , 0.6, 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. ]])

In [4]:
react_yield_p0

array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3])

In [5]:
sputter_yield_coefficient

[0.3, 0.001, 0.7853981633974483]

In [ ]:
particle_ratio = (1 - 0.08)

In [ ]:
9+11+3+1